In [3]:
!pip install neo4j

  Using cached neo4j-5.26.0-py3-none-any.whl.metadata (5.9 kB)
Using cached neo4j-5.26.0-py3-none-any.whl (302 kB)


In [11]:
from neo4j import GraphDatabase, Driver, Result, Session
import configparser

In [12]:
config = configparser.ConfigParser()
config.read('./graph_api.config')

URI = "neo4j+s://hackatum-one.graphdatabase.ninja:443"
AUTH = (config.get('AUTH', 'username'), config.get('AUTH', 'password'))

drv: Driver = GraphDatabase.driver(uri=URI, auth=AUTH, max_connection_lifetime=60)
drv.database = "attendee14"
sess: Session = drv.session()
res: Result = sess.run(query="MATCH (n) RETURN n LIMIT 1")
print(res.single())

<Record n=<Node element_id='4:d0c5e6c7-6850-48df-9900-af21060ab9c0:0' labels=frozenset({'Hostname'}) properties={'label': 'humidifies.flickering.statuabamus.net', 'key': 'urn:dns:humidifies.flickering.statuabamus.net'}>>


In [19]:
from neo4j import GraphDatabase
import pandas as pd

def fetch_affected_software_inst(uri, auth, database, product, min_version, max_version):
    try:
        drv = GraphDatabase.driver(uri=uri, auth=auth, max_connection_lifetime=60)
        drv.database = database

        with drv.session() as sess:
            query = """
            MATCH (n:SoftwareInstallation)
            WHERE 
              (n.product CONTAINS $product AND n.version >= $min_version AND n.version <= $max_version)
            RETURN n
            """
            result = sess.run(query, product=product, min_version=min_version, max_version=max_version)

            # Fetch results into a list of dictionaries
            records = [record["n"] for record in result]

        # Convert to DataFrame for tabular display
        df = pd.DataFrame(records)
        print(df.to_string(index=False))  # Display table without index

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        drv.close()

In [20]:
# Docker Example
URI = "neo4j+s://hackatum-one.graphdatabase.ninja:443"
AUTH = ("attendee14", "EDUZB71$JF")
DATABASE = "attendee14"

fetch_affected_software_inst(
    uri=URI,
    auth=AUTH,
    database=DATABASE,
    product="Docker",
    min_version="23.0",
    max_version="26.1.3"
)

  edition                                       key       product publisher version
Community cpe:/a:docker:docker_engine:24::community Docker Engine    Docker      24
Community    cpe:/a:docker:docker_cli:24::community    Docker CLI    Docker      24


In [24]:
def count_affected_systems(uri, auth, database, product, min_version, max_version):
    try:
        drv = GraphDatabase.driver(uri=uri, auth=auth, max_connection_lifetime=60)
        drv.database = database

        with drv.session() as sess:
            query = """
            MATCH (n:SoftwareInstallation)
            WHERE 
              (n.product CONTAINS $product AND n.version >= $min_version AND n.version <= $max_version)
            MATCH (n)-[:related_software]-(system:System)
            RETURN COUNT(DISTINCT system) AS systemCount
            """
            result = sess.run(query, product=product, min_version=min_version, max_version=max_version)

            # Fetch the count from the result
            system_count = result.single()["systemCount"]
            print(f"Number of connected systems: {system_count}")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        drv.close()

In [25]:
count_affected_systems(
    uri=URI,
    auth=AUTH,
    database=DATABASE,
    product="Docker",
    min_version="23.0",
    max_version="26.1.3"
)

Number of connected systems: 202


In [28]:
def fetch_affected_systems(uri, auth, database, product, min_version, max_version):
    try:
        drv = GraphDatabase.driver(uri=uri, auth=auth, max_connection_lifetime=60)
        drv.database = database

        with drv.session() as sess:
            query = """
            MATCH (n:SoftwareInstallation)
            WHERE 
              (n.product CONTAINS $product AND n.version >= $min_version AND n.version <= $max_version)
            MATCH (n)-[:related_software]-(system:System)
            RETURN DISTINCT system
            """
            result = sess.run(query, product=product, min_version=min_version, max_version=max_version)

            records = [record["system"] for record in result]

        if records:
            df = pd.DataFrame(records)
            print(df.to_string(index=False))
        else:
            print("No affected systems found.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        drv.close()

In [29]:
fetch_affected_systems(
    uri=URI,
    auth=AUTH,
    database=DATABASE,
    product="Docker",
    min_version="23.0",
    max_version="26.1.3"
)

 critical        id                  key  provider_name        state        sub_type    type
        1 266540047 urn:system:266540047       Homebrew       Active           Other Unknown
        0 266495165 urn:system:266495165       Homebrew       Active           Other  Server
        0 266467486 urn:system:266467486       Homebrew       Active  Virtual Server  Server
        1 266611090 urn:system:266611090       Homebrew       Active           Other Unknown
        0 266668102 urn:system:266668102       Homebrew       Active           Other  Server
        1 266778190 urn:system:266778190        Helheim Inconsistent  Virtual Server  Server
        0 266693977 urn:system:266693977  MagnaCumLaude       Active           Other Unknown
        0 266693976 urn:system:266693976  MagnaCumLaude       Active           Other Unknown
        0 266958572 urn:system:266958572       Homebrew Inconsistent           Other  Server
        1 266485431 urn:system:266485431       Homebrew       Active  

In [72]:
def find_responsible_person(uri, auth, database, product, min_version, max_version):
    try:
        drv = GraphDatabase.driver(uri=uri, auth=auth, max_connection_lifetime=60)
        drv.database = database

        with drv.session() as sess:
            query = """
            MATCH (n:SoftwareInstallation)
            WHERE 
              (n.product CONTAINS $product AND n.version >= $min_version AND n.version <= $max_version)
            MATCH (n)-[:related_software]-(system:System)
            MATCH (system)-[:runs_on]-(app:Application)
            MATCH (app)-[:owned_by]-(org:OrgUnit)
            MATCH (org)-[:head_of]-(person:Person)
            RETURN DISTINCT system, person, org, app
            """
            result = sess.run(query, product=product, min_version=min_version, max_version=max_version)

            records = [{"system": record["system"], "person": record.get("person", None), 
                        "application": record.get("app", None), "org_unit": record.get("org", None)} 
                       for record in result]

        responsible_data = []
        for record in records:
            system_id = record["system"].id if record["system"] else None
            person_name = record["person"].get("fullname") if record["person"] else "No responsible person"
            app_name = record["application"].get("name") if record["application"] else "No application"
            org_name = record["org_unit"].get("orgCode") if record["org_unit"] else "No org unit"
            
            responsible_data.append({
                "System ID": system_id,
                "Application": app_name,
                #"Org Unit": org_name,
                "Responsible Person": person_name
            })

        df = pd.DataFrame(responsible_data)
        print(df.to_string(index=False))

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        drv.close()

In [73]:
find_responsible_person(
    uri=URI,
    auth=AUTH,
    database=DATABASE,
    product="Docker",
    min_version="23.0",
    max_version="26.1.3"
)

 System ID                              Application       Responsible Person
   7499018             Tangerine yellow Ouagadougou           Brett Nakayama
   7502853                     Antique white Lisbon Iraida Sigurbjörnsdóttir
   7502853                     Antique white Lisbon         Sarasati Baquero
   7502853                     Antique white Lisbon          Krisztián Bonsu
   3852478                                        '              Karola Caro
   3852478                        Raspberry Colombo          Domitila Quiroz
   3852478                          Cornflower Dili          Domitila Quiroz
   3852478                              Gray Ankara              Márton Meza
   3852478 Pansy purple Edinburgh of the Seven Seas            Danilo Motoki
   3852478                         Moccasin Palikir     Angel Sigursveinsson
   7495662             Tangerine yellow Ouagadougou           Brett Nakayama
   3848469                    Lincoln green Kampala            Lhaba Sanjuan

/var/folders/w_/0j5zvgc122d9lh7r90c5xg540000gn/T/ipykernel_16296/3521279229.py:25: DeprecationWarning: `id` is deprecated, use `element_id` instead
  system_id = record["system"].id if record["system"] else None


In [81]:
def try_fetch_epss(uri, auth, database, cve_id):
    try:
        drv = GraphDatabase.driver(uri=uri, auth=auth, max_connection_lifetime=60)
        drv.database = database

        with drv.session() as sess:
            query = """
            MATCH (cve:Vulnerability)
            WHERE cve.cve = $cve_id
            MATCH (cve)-[:has_rating]->(rating:EPSS)
            RETURN cve.cve AS CVE_ID, rating.score AS EPSS_Score
            """
            result = sess.run(query, cve_id=cve_id)
            
            records = [record for record in result]
            if not records:
                print(f"{cve_id} not found in the database.")
                return None

            df = pd.DataFrame(records, columns=["CVE_ID", "EPSS_Score"])
            return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        drv.close()

In [82]:
cve_data = try_fetch_epss(uri=URI, auth=AUTH, database=DATABASE, cve_id="CVE-2019-8033")
if cve_data is not None:
    print(cve_data)

          CVE_ID  EPSS_Score
0  CVE-2019-8033     0.01393
